## Para autores
query = '''
CALL gds.nodeSimilarity.stream(
  'authorPaper', 
  {
    relationshipTypes: ['keyWords','author','affiliation'] 
  }
)
YIELD node1, node2, similarity
WITH gds.util.asNode(node1) AS n1, gds.util.asNode(node2) AS n2, similarity
WHERE 'Author' IN labels(n1) 
  AND 'Author' IN labels(n2)  
MERGE (n1)-[r:AUTHOR_SIMILARITY]->(n2)
SET r.similarity = similarity
'''


result = graph.run(query)

## Para papers 


query = '''
CALL gds.nodeSimilarity.stream(
  'paperGraph', 
  {
    relationshipTypes: ['keyWords']
  }
)
YIELD node1, node2, similarity
WITH gds.util.asNode(node1) AS n1, gds.util.asNode(node2) AS n2, similarity
WHERE 'Paper' IN labels(n1) 
  AND 'Paper' IN labels(n2) 
MERGE (n1)-[r:SIMILAR_BY_KEY]->(n2)
SET r.similarity = similarity
'''

result = graph.run(query)


In [9]:
import streamlit as st
import py2neo
import pandas as pd

nuevaLabel="Artificial intelligence"
graph = py2neo.Graph("bolt://localhost:7687",user = "neo4j" ,password="123456789", name = "tesis")

def label(graph, entity1, label1):
    # Formar la consulta con la etiqueta (no puede ser parámetro)
    query = f'''
        MATCH (e:{entity1})<-[:hasSubject|:hasObject]-(f:Fact)<-[:hasPart]-(c:Chunk)<-[:hasPart]-(p:Paper)<-[:author]-(a:Author)
        WITH e.label AS Entidad, p.identifier AS PaperID
        WITH Entidad, COUNT(DISTINCT PaperID) AS Frecuencia
        ORDER BY Frecuencia DESC
        RETURN Entidad
        Limit 10

        '''
    # Ejecutar la consulta
    return graph.run(query, label1=label1) 

SyntaxError: invalid syntax (3753297120.py, line 4)

In [5]:
print(label(graph,"Method","Cluster analysis"))

(No data)


In [6]:
query = '''
CALL gds.graph.list()
YIELD graphName
CALL gds.graph.drop(graphName)
YIELD graphName AS droppedGraph
RETURN droppedGraph
'''

graph.run(query)


query = '''
CALL gds.graph.project(
  'paperGraph',
  ['Paper', 'Fact', 'Chunk', 'Key', 'Technology', 'Method', 'Person', 'Group', 'Algoritm', 'ResearchField', 'Tool','Evaluation'
  ,'DataSet', 'Exprement'],
  {
    keyWords: {
      orientation: 'UNDIRECTED'
    },
    RELATED_TO: {
      orientation: 'UNDIRECTED'
    }
    
      
  }
)
'''
graph.run(query) 
query = '''
CALL gds.graph.project(
  'authorPaper',
  ['Key','Paper', 'Author','Affiliation'],
  {
    author: {
      orientation: 'UNDIRECTED'
    },
    affiliation: {
      orientation: 'UNDIRECTED'
    },
    keyWords: {
      orientation: 'UNDIRECTED'
    }
  }
)
'''
graph.run(query) 

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{Paper: {label: 'Paper', properties: {}}, Affiliation: {label: 'Affiliation', properties: {}}, Key: {label: 'Key', properties: {}}, Author: {label: 'Author', properties: {}}}","{author: {aggregation: 'DEFAULT', orientation: 'UNDIRECTED', indexInverse: false, properties: {}, type: 'author'}, keyWords: {aggregation: 'DEFAULT', orientation: 'UNDIRECTED', indexInverse: false, properties: {}, type: 'keyWords'}, affiliation: {aggregation: 'DEFAULT', orientation: 'UNDIRECTED', indexInverse: false, properties: {}, type: 'affiliation'}}",authorPaper,288028,525680,62


Author

In [28]:
query = '''
CALL gds.nodeSimilarity.stream(
  'authorPaper', 
  {
    relationshipTypes: ['keyWords','author','affiliation'] 
  }
)
YIELD node1, node2, similarity
WITH gds.util.asNode(node1) AS n1, gds.util.asNode(node2) AS n2, similarity
WHERE 'Author' IN labels(n1) 
  AND 'Author' IN labels(n2) 
  
RETURN n1.identifier AS node1,n1.name, n2.identifier AS node2,n2.name, similarity
ORDER BY similarity DESC,node1, node2
'''


result = graph.run(query)

In [7]:
query = '''
CALL gds.nodeSimilarity.stream(
  'authorPaper', 
  {
    relationshipTypes: ['keyWords','author','affiliation'] 
  }
)
YIELD node1, node2, similarity
WITH gds.util.asNode(node1) AS n1, gds.util.asNode(node2) AS n2, similarity
WHERE 'Author' IN labels(n1) 
  AND 'Author' IN labels(n2)  
MERGE (n1)-[r:AUTHOR_SIMILARITY]->(n2)
SET r.similarity = similarity
'''


result = graph.run(query)

In [29]:
dataSim = result.data()
df = pd.DataFrame(dataSim)
df

,node1,n1.name,node2,n2.name,similarity
0,10038907100,"Qiu, Zhong Pan",16202478800,"Han, Shuihua",1.000000
1,10038907100,"Qiu, Zhong Pan",57328044300,"Sandrine, Mukase",1.000000
2,10039124600,"Ryan, Sadie J.",13005665700,"Blackburn, Jason K.",1.000000
3,10039124600,"Ryan, Sadie J.",19336815200,"Burkett-Cadena, Nathan D.",1.000000
4,10039124600,"Ryan, Sadie J.",35772939700,"Mao, Liang",1.000000
...,...,...,...,...,...
246946,57219659228,"Soto, Carlos Manuel Diaz",56267555100,"Guerrero, Janio Jadan",0.009259
246947,16743568200,"Ferrás, Carlos",53866250100,"Guarda, T.",0.005348
246948,16743568200,"Ferrás, Carlos",53866250100,"Guarda, Teresa",0.005348
246949,57202507497,"Haro, Marco Salinas",53866250100,"Guarda, T.",0.005348


KEYWORDS

In [132]:

query = '''
CALL gds.nodeSimilarity.stream(
  'paperGraph', 
  {
    relationshipTypes: ['keyWords']
  }
)
YIELD node1, node2, similarity
WITH gds.util.asNode(node1) AS n1, gds.util.asNode(node2) AS n2, similarity
WHERE n1.identifier = '2-s2.0-85075203274'
  AND 'Paper' IN labels(n2) 
RETURN n1.identifier AS node1, n2.identifier AS node2, similarity
ORDER BY similarity DESC,node1, node2
'''

result = graph.run(query)

In [8]:

query = '''
CALL gds.nodeSimilarity.stream(
  'paperGraph', 
  {
    relationshipTypes: ['keyWords']
  }
)
YIELD node1, node2, similarity
WITH gds.util.asNode(node1) AS n1, gds.util.asNode(node2) AS n2, similarity
WHERE 'Paper' IN labels(n1) 
  AND 'Paper' IN labels(n2) 
MERGE (n1)-[r:SIMILAR_BY_KEY]->(n2)
SET r.similarity = similarity
'''

result = graph.run(query)

In [21]:

dataSim = result.data()
df = pd.DataFrame(dataSim)
df

""


Por Entity

In [16]:

query = '''
CALL gds.nodeSimilarity.stream(
  'paperGraph', 
  {
    relationshipTypes: ['RELATED_TO']
  }
)
YIELD node1, node2, similarity
WITH gds.util.asNode(node1) AS n1, gds.util.asNode(node2) AS n2, similarity
WHERE n1.identifier = '2-s2.0-85075203274'
RETURN n1.identifier AS node1, n2.identifier AS node2, similarity AS similarity
ORDER BY similarity DESC, node2
'''


result = graph.run(query)

In [17]:
dataSim = result.data()
df = pd.DataFrame(dataSim)
df

,node1,node2,similarity
0,2-s2.0-85075203274,2-s2.0-85096234507,0.054945
1,2-s2.0-85075203274,2-s2.0-85114651956,0.053030
2,2-s2.0-85075203274,2-s2.0-85068836918,0.050000
3,2-s2.0-85075203274,2-s2.0-85096435287,0.046667
4,2-s2.0-85075203274,2-s2.0-85109310570,0.041667
5,2-s2.0-85075203274,2-s2.0-85194862356,0.040724
6,2-s2.0-85075203274,2-s2.0-85130294087,0.035461
7,2-s2.0-85075203274,2-s2.0-85052641362,0.035088
8,2-s2.0-85075203274,2-s2.0-85119413504,0.031008
9,2-s2.0-85075203274,2-s2.0-85077504814,0.029740


droppedGraph
paperGraph
authorPaper


In [2]:
query = '''
MATCH (p:Paper)-[:hasPart*2]->(f:Fact)-[:hasSubject|:hasObject]->(e)
MERGE (p)-[:RELATED_TO]->(e);
'''

result = graph.run(query)
result

(No data)